In [1]:
%cd ..


/mnt/c/Users/maxli/OneDrive - Van Oord/Documents/thesis/code


In [2]:
from atl_module.geospatial_utils.raster_interaction import random_raster_gdf,subset_gebco
from atl_module import GebcoUpscaler
from atl_module.kriging import krige_bathy
from atl_module.error_calc import raster_RMSE_blocked
from atl_module.kalman import gridded_kalman_update
import geopandas as gpd

In [3]:
truthraster_path = '../data/special_sites/niiahu/in-situ-dem/truth.vrt'

gdf = random_raster_gdf(truthraster_path,3000)
gdf.to_file('../data/truthraster_sampling/all_bathy_pts.gpkg')
gdf = gdf.to_crs(gdf.estimate_utm_crs())

In [4]:
gdf

,truth raster elevation,geometry
0,5.047521,POINT (385885.982 2432601.007)
1,-44.337727,POINT (374398.555 2425409.900)
2,-5.073222,POINT (381000.961 2426891.467)
3,-20.117380,POINT (375877.316 2426399.032)
4,-29.603466,POINT (379894.561 2428962.461)
...,...,...
2995,-9.569372,POINT (376808.974 2425590.905)
2996,-16.698721,POINT (376409.478 2426291.843)
2997,7.522849,POINT (391179.461 2431614.706)
2998,3.500359,POINT (391122.133 2432283.574)


In [5]:
hres = 50
subset_gebco('../data/truthraster_sampling',bathy_pts=gdf,epsg_no=gdf.crs.to_epsg(),hres=hres)

[D 220918 15:51:01 raster_interaction:128] GEBCO subset raster written to ../data/truthraster_sampling/bilinear.tif, with CRS EPSG:32604


In [6]:
gdf = gdf.rename(columns={'truth raster elevation':'sf_elev_MSL'})

In [7]:
gdf = gdf[gdf.sf_elev_MSL < 0]


In [ ]:
import rasterio 
import matplotlib.pyplot as plt
from rasterio.plot import show as rastershow

with rasterio.open('../data/special_sites/niiahu/in-situ-dem/Job761532_usace2013_niihau_dem.tif') as femaras:
    fig, ax = plt.subplots(figsize=(20, 12))
    ax.set_xlabel(f"Degrees longitude in {femaras.crs}")
    ax.set_ylabel(f"Degrees latitude in {femaras.crs}")
    # ax.set_title("")
    # cx.add_basemap(ax,source=cx.providers.OpenTopoMap,crs=femaras.crs)
    image_hidden = ax.imshow(
        femaras.read(1, masked=True),
        cmap="inferno",
    )
    rastershow(femaras, cmap="inferno", ax=ax)

    fig.colorbar(image_hidden, ax=ax)

fig.savefig('../document/figures/truthraster_sampling_rasterplot.jpg',dpi=500,bbox_inches='tight')

In [ ]:
gdf_sampled = gpd.read_file('../data/truthraster_sampling/kriging_pts/')


In [ ]:
ax = gdf_sampled.plot(figsize=(20, 12))
ax.get_figure().savefig('../document/figures/pts_sampled_from_truthraster.jpg')

In [ ]:
krige_bathy(kr_model='uk',folderpath='../data/truthraster_sampling',npts=1500,variogram_model='spherical',pts_gdf_all=gdf,crs=gdf.crs,samplemethod='dart', variogram_parameters={'range':20000,'nugget':0.001,'sill':23},
)
# .kriging(1500,kr_model='uk',samplemethod = 'dart')

In [ ]:
bilinear_rmse = raster_RMSE_blocked(truth_raster_path=truthraster_path,measured_rasterpath='../data/truthraster_sampling/bilinear.tif')

In [ ]:
purekriged_rmse = raster_RMSE_blocked(truth_raster_path=truthraster_path,measured_rasterpath='../data/truthraster_sampling/kriging_output.tif')

In [ ]:
gridded_kalman_update('../data/truthraster_sampling/kalman_update.tif','../data/truthraster_sampling/bilinear.tif','../data/truthraster_sampling/kriging_output.tif',3)
kalman_updated_rmse =raster_RMSE_blocked(truth_raster_path=truthraster_path,measured_rasterpath='../data/truthraster_sampling/kalman_update.tif')

In [ ]:
import pandas as pd
rmse_dict = {
    f'Truth vs {hres}m bilinear resampling of GEBCO':bilinear_rmse,
             'Truth vs Kriged raster output':purekriged_rmse,
             'Truth vs GEBCO+Kriged raster':kalman_updated_rmse
            }
truthraster_sampling_table = pd.DataFrame.from_dict(rmse_dict,orient='columns').T

In [ ]:
truthraster_sampling_table.style.to_latex('../document/tables/truth_raster_sample_error.tex',caption='Error between the various data products',label='tab:rmse-truth-raster-sampled')

In [ ]:
truthraster_sampling_table